In [74]:
import os                             #### FOR MULTIPLE FEATURES X, SINGLE Y ####
import sys
import tensorflow as tf

W = tf.Variable(tf.zeros([2,1]), name="weights")
b = tf.Variable(0., name="bias")

In [108]:
# ------- define the training loop operations -------

def inference(X): # compute inference model over data X and return the result
    return tf.matmul(X, W) + b

def loss(X, Y): # compute loss over training data X and expected outputs Y
    Y_predicted = inference(X)
    return tf.reduce_sum(tf.squared_difference(Y, Y_predicted))

def inputs(): # read/generate input training data X and expected outputs Y    
    # Data from http://people.sc.fsu.edu/~jburkardt/datasets/regression/x09.txt
    weight_age = [[84, 46], [73, 20], [65, 52], [70, 30], [76, 57], [69, 25], [63, 28], [72, 36], [79, 57], 
                  [75, 44], [27, 24], [89, 31], [65, 52], [57, 23], [59, 60], [69, 48], [60, 34], [79, 51], 
                  [75, 50], [82, 34], [59, 46], [67, 23], [85, 37], [55, 40], [63, 30]]
    blood_fat_content = [354, 190, 405, 263, 451, 302, 288, 385, 402, 365, 209, 290, 346, 254, 395, 434, 220, 
                         374, 308, 220, 311, 181, 274, 303, 244]
    return tf.to_float(weight_age), tf.to_float(blood_fat_content) # X, Y
    
def train(total_loss): # train/adjust model parameters according to computed total loss
    learning_rate = 0.0000001
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)
    
def evaluate(sess, X, Y): # evaluate the resulting trained model
    print(sess.run(inference([[80., 25.]]))) # 303
    print(sess.run(inference([[65., 25.]]))) # 256

# ---------------------------------------------------

In [123]:
#saver = tf.train.Saver() # by default the saver will keep only the most recent 5 files and delete the rest
#currentDir = 'C:\\Users\\Thuy Nguyen\\Desktop\\Tensorflow learning\\my-model'

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    X, Y = inputs()
    #print(sess.run(X)) # USE sess TO PRINT VALUES TO SCREEN
    
    total_loss = loss(X, Y)
    train_op = train(total_loss)    
    
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    
    initial_step = 0
    
    # verify if we don't have a checkpoint saved already
    #check_point = tf.train.get_checkpoint_state(currentDir) 
    #if check_point and check_point.model_checkpoint_path:
    #    saver.restore(sess, check_point.model_checkpoint_path)
    #    initial_step = int(check_point.model_checkpoint_path.rsplit('-', 1)[1])
    
    # actual training loop
    training_steps = 1000
    for step in range(initial_step, training_steps):
        sess.run([train_op])
        if step % 100 == 0: # for debug purpose, see how the loss gets decremented thru training steps
            print("loss: ", sess.run([total_loss]))
            #saver.save(sess, currentDir, global_step=step) # creates a checkpoint file with the name template
                                                           # my-model-{step} like my-model-1000, my-model-2000...
    evaluate(sess, X, Y)
    
    #saver.save(sess, currentDir, global_step=training_steps)
    
    coord.request_stop()
    coord.join(threads)
    sess.close()

loss:  [7608772.5]
loss:  [5341925.5]
loss:  [5339993.0]
loss:  [5338747.0]
loss:  [5337538.5]
loss:  [5336334.0]
loss:  [5335129.5]
loss:  [5333926.0]
loss:  [5332724.0]
loss:  [5331524.0]
[[ 320.64968872]]
[[ 267.78182983]]
